<a href="https://colab.research.google.com/github/stoicM/stoicM/blob/main/InvokeAI_LoRA_Metadata_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 InvokeAI + LoRA Setup with Metadata Logging
Includes:
- Google Drive Mount
- Custom Filename Templates
- LoRA Load/Save Support
- Metadata Logging for Prompt, Seed, Model, Filename

In [ ]:
# 📂 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🗂️ Set up working directories
import os
base_path = '/content/drive/MyDrive/InvokeAI'
output_dir = f'{base_path}/images/generated'
lora_dir = f'{base_path}/LoRA'
meta_log_path = f'{base_path}/metadata/meta_log.csv'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(lora_dir, exist_ok=True)
os.makedirs(os.path.dirname(meta_log_path), exist_ok=True)
!echo 'Folders ready.'

In [ ]:
# ⚙️ Install dependencies
!pip install -q diffusers transformers accelerate safetensors gradio pandas ngrok

In [ ]:
# 🔄 Load Model
from diffusers import DiffusionPipeline
pipe = DiffusionPipeline.from_pretrained(
    'John6666/wai-nsfw-illustrious-v90-sdxl',
    torch_dtype='auto'
).to('cuda')

In [ ]:
# 🎛️ UI for generation, filename tagging, and LoRA load (simulated)
import gradio as gr
from PIL import Image
import time, random
import pandas as pd

def generate(prompt, pose_tag, use_seed, custom_seed, filename_prefix, lora_file):
    seed = custom_seed if use_seed else random.randint(1000000, 9999999)
    generator = pipe.generator.manual_seed(seed)
    image = pipe(prompt, generator=generator).images[0]

    timestamp = time.strftime('%Y%m%d-%H%M%S')
    filename = f"{filename_prefix}_{pose_tag}_{seed}_{timestamp}.png"
    save_path = os.path.join(output_dir, filename)
    image.save(save_path)

    df = pd.DataFrame([[filename, prompt, seed, 'wai-nsfw-illustrious-v90-sdxl', pose_tag, timestamp]],
                      columns=['filename', 'prompt', 'seed', 'model', 'pose_tag', 'timestamp'])
    if os.path.exists(meta_log_path):
        df.to_csv(meta_log_path, mode='a', header=False, index=False)
    else:
        df.to_csv(meta_log_path, index=False)

    return image

gr.Interface(
    fn=generate,
    inputs=[
        gr.Textbox(label='Prompt'),
        gr.Textbox(label='Pose Tag (e.g., pose1, 2_3qrt)'),
        gr.Checkbox(label='Use Custom Seed?'),
        gr.Number(label='Seed (if checked)', value=123456),
        gr.Textbox(label='Filename Prefix', value='alice'),
        gr.Textbox(label='LoRA File (future use)', placeholder='e.g., alice_lora_v1.safetensors')
    ],
    outputs='image',
    title='Alice Gen – InvokeAI w/ Metadata + LoRA Ready'
).launch(share=True)